In [16]:
%%writefile requirements.txt
selenium==3.141.0 
urllib3==1.26.16

Writing requirements.txt


In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [6]:
edge_options = {
    "executable_path": "./msedgedriver",
    # ofcourse change path to driver you downloaded.
    "capabilities": {
        "platformName": 'mac os x', # I get this from Chrome driver's capabilities
        # "os" : "OS X", # also ok.
    }
}
driver = webdriver.Edge(**edge_options)

In [7]:
driver.get("https://xxx.com/")

# log in manually into the profile

## Javascript code to run in browser console to figure out the coordinates of the seat

In [ ]:
var canvas = document.querySelector('canvas');
canvas.addEventListener('click', function(event) {
    var rect = canvas.getBoundingClientRect();
    var x = event.clientX - rect.left;
    var y = event.clientY - rect.top;
    console.log('Clicked coordinates: x=' + x + ', y=' + y);
});

In [8]:
# translate month to number
month_text2no = {"January": 1,
                "February": 2,
                "March": 3,
                "April": 4,
                "May": 5,
                "June": 6,
                "July": 7,
                "August": 8,
                "September": 9,
                "October": 10,
                "November": 11,
                "December": 12
}
shortmonth_text2no = {"Jan": 1,
                "Feb": 2,
                "Mar": 3,
                "Apr": 4,
                "May": 5,
                "Jun": 6,
                "Jul": 7,
                "Aug": 8,
                "Sep": 9,
                "Oct": 10,
                "Nov": 11,
                "Dec": 12
}

time_slots = {
    'am':[
        '09:00 - 09:30',
        '09:30 - 10:00',
        '10:00 - 10:30',
        '10:30 - 11:00',
        '11:00 - 11:30',
        '11:30 - 12:00',
        '12:00 - 12:30',
        '12:30 - 13:00',
    ],
    'pm':[
        '14:00 - 14:30',
        '14:30 - 15:00',
        '15:00 - 15:30',
        '15:30 - 16:00',
        '16:00 - 16:30',
        '16:30 - 17:00',
        '17:00 - 17:30',
    ]
}
tables_cood = {
    5: [180,208],
    8: [186,324],
    9: [207,207],
    12: [216,327],
    13: [128, 393], # my seat
    14: [172, 413],
    15: [189, 380],
    16: [136, 440], # yy
    20: [380, 350],
    22: [370, 432],
    23: [396, 432],
}

# Run the following
- After you have log into the website manually
- After you have confirmed the coordinates of the seats

In [9]:
from datetime import datetime, timedelta

def get_booking_dates(start_date):
    # Define the start date and the end date (3 months from the start date)
    start_date = datetime.strptime(start_date, "%b %d, %Y")
    end_date = start_date + timedelta(days=3*30)  # Approximation for 3 months

    # Initialize the current date to start date
    current_date = start_date

    # Initialize a list to store the dates
    booking_dates = []

    # Iterate over the dates from start to end
    while current_date <= end_date:
        # Get the weekday (0 is Monday, 1 is Tuesday, 2 is Wednesday)
        if current_date.weekday() <= 2:  # Monday, Tuesday, or Wednesday
            # Append the date in the specified format
            booking_dates.append(current_date.strftime("%b %-d, %Y"))
        # Move to the next day
        current_date += timedelta(days=1)

    return booking_dates


In [10]:
table = int(input("which table do you want to book?"))
startdate = input("Enter booking start date in this format Apr 1, 2024")
booking_dates = get_booking_dates(startdate)

failed_dates = []
for date in booking_dates[:]:
    for ampm in ['am', 'pm']:
        target_date = date
        target_mth = shortmonth_text2no[target_date[:3]]

        driver.get("https://xxx.com/Book-a-Space")
        time.sleep(1)
        # click on Do some quiet work
        driver.find_element(By.CSS_SELECTOR,'button[id="Do some quiet work"]').send_keys(Keys.RETURN)
        time.sleep(1)
        # click on Next
        driver.find_element(By.CSS_SELECTOR,'button[class*="w-full h-full py-4 px-5 txt-MontserratExtraBold"]').send_keys(Keys.RETURN)
        time.sleep(1)
        # check current default month
        current_month = driver.find_element(By.CSS_SELECTOR,'div[class*="css-1v994a0"]').text
        month_text2no[current_month]
        time.sleep(0.5)

        # press the next month button depending on how many times it is away from our target_mth
        for i in range(target_mth-month_text2no[current_month]):
            driver.find_element(By.CSS_SELECTOR,'button[title="Next month"]').send_keys(Keys.RETURN)
        time.sleep(0.75)

        try:
            # press the date
            driver.find_element(By.CSS_SELECTOR,f'button[aria-label="{target_date}"]').send_keys(Keys.RETURN)
            time.sleep(0.5)
            
            # click on Next
            driver.find_element(By.CSS_SELECTOR,'button[class*="w-full h-full py-4 px-5 txt-MontserratExtraBold"]').send_keys(Keys.RETURN)
            time.sleep(0.5)

            # Select AM or PM time slot. 
            for time_slot in time_slots[ampm]:
                selected_time_slot = driver.find_elements_by_xpath(f"//*[contains(text(), '{time_slot}')]")[0].send_keys(Keys.RETURN)
            time.sleep(0.5)

            # click on Next
            driver.find_element(By.CSS_SELECTOR,'button[class*="w-full h-full py-4 px-5 txt-MontserratExtraBold"]').send_keys(Keys.RETURN)
            time.sleep(1.5)

            # try selecting seat
            try:
                js_script = f"""
                var stage = Konva.stages[0];  // Get the first Konva Stage

                // Get the shape at the current position
                var shape = stage.getIntersection({{x: {tables_cood[table][0]}, y: {tables_cood[table][1]}}});

                if (shape) {{
                    // Simulate a click on the shape
                    shape.fire('click');
                    // return "Clicked at (" + 160 + ", " + 380 + ")";
                }}
                """
                output = driver.execute_script(js_script)
                time.sleep(1)

                # click on Book
                driver.find_element(By.CSS_SELECTOR,'button[class*="w-full h-full py-4 px-5 txt-MontserratExtraBold"]').send_keys(Keys.RETURN)
                time.sleep(1)

                # Confirm booking by clicking on the 2nd button
                driver.find_elements(By.CSS_SELECTOR,'button[class*="w-full h-full py-4 px-5 txt-MontserratExtraBold"]')[1].send_keys(Keys.RETURN)
                time.sleep(6)

                # confirm
                driver.find_elements(By.CSS_SELECTOR,'button[class*="w-full h-full py-4 px-5 txt-MontserratExtraBold"]')[1].send_keys(Keys.RETURN)
                time.sleep(1)
                print(f"Booking table {table} on {target_date}, {ampm} successfully")
            except:
                print(f"Unable to book for {target_date}, {ampm}. Table {table} not available for booking")
                failed_dates.append(target_date + ' ' + ampm)
        
        except:
            print(f"Unable to book for {target_date}, {ampm}. Date not available for booking.")
            failed_dates.append(target_date + ' ' + ampm)
            pass

Unable to book for Oct 1, 2024, am. Table 13 not available for booking
Unable to book for Oct 1, 2024, pm. Table 13 not available for booking
Booking table 13 on Oct 2, 2024, am successfully
Booking table 13 on Oct 2, 2024, pm successfully
Booking table 13 on Oct 7, 2024, am successfully
Booking table 13 on Oct 7, 2024, pm successfully
Booking table 13 on Oct 8, 2024, am successfully
Booking table 13 on Oct 8, 2024, pm successfully
Booking table 13 on Oct 9, 2024, am successfully
Booking table 13 on Oct 9, 2024, pm successfully
Booking table 13 on Oct 14, 2024, am successfully
Booking table 13 on Oct 14, 2024, pm successfully
Booking table 13 on Oct 15, 2024, am successfully
Booking table 13 on Oct 15, 2024, pm successfully
Booking table 13 on Oct 16, 2024, am successfully
Booking table 13 on Oct 16, 2024, pm successfully
Booking table 13 on Oct 21, 2024, am successfully
Booking table 13 on Oct 21, 2024, pm successfully
Booking table 13 on Oct 22, 2024, am successfully
Booking table 13

KeyboardInterrupt: 